In [ ]:
import stackstac
import pystac_client
import numpy as np
import xarray as xr
import rioxarray
import rasterio
from affine import Affine
import geopandas as gpd
import requests
from IPython.display import display
from PIL import Image
from ipywidgets import widgets
from sklearn.ensemble import RandomForestClassifier
from ipyleaflet import Map,GeoJSON, ImageOverlay
import matplotlib
from base64 import b64encode
from io import BytesIO
from skimage.morphology import binary_erosion, binary_dilation

In [ ]:
roi_gdf = gpd.read_file("../res/roi.geojson")

map = Map(center=tuple(roi_gdf.geometry[0].centroid.coords)[0][::-1], zoom=9)
map.add(
    GeoJSON(data=roi_gdf.geometry[0].__geo_interface__, style={"color": "red"})
)

In [ ]:
URL = "https://earth-search.aws.element84.com/v1"
catalog = pystac_client.Client.open(URL)

items = catalog.search(
    intersects=roi_gdf.geometry[0],
    collections=["sentinel-2-l2a"],
    datetime="2015-03-01/2023-06-01"
).item_collection()

len(items)

In [ ]:
nocloud_items = [
    item for item in items
    if (
        item.properties["eo:cloud_cover"] < 5
    )
]
len(nocloud_items)

Let's visualize the queried products using the preview pictures that come with them


In [ ]:
widgets.GridBox(
    children = [
        widgets.VBox(
            children = [
                widgets.Label(item.id),
                widgets.Image(value=requests.get(item.assets["thumbnail"].href, stream=True).content)
            ],
            layout=widgets.Layout(border='solid',)
        )
        for item in nocloud_items[::5]
    ],
    layout=widgets.Layout(
        grid_template_columns='256px ' * 4,
    )
)

In [ ]:
items[0].properties['s2:water_percentage'] / (
    items[0].properties['s2:vegetation_percentage'] +
    items[0].properties['s2:not_vegetated_percentage'] +
    items[0].properties['s2:water_percentage']
)
items[0].properties

In [ ]:
def preview_miniature_on_map(item):
    t = item.assets["blue"].extra_fields["proj:transform"]
    da = xr.DataArray(
        data=np.array(Image.open(requests.get(item.assets["thumbnail"].href, stream=True).raw)),
        dims = ["y", "x", "band"],
    ).transpose('band', 'y', 'x').rio.write_crs(item.properties["proj:epsg"]).rio.write_transform(
        Affine(10 * 10980 / 343, *t[1:4], -10 * 10980 / 343, t[5])
    ).rio.reproject("epsg:4326")
    im = Image.fromarray(da.transpose('y', 'x', 'band').values)

    map = Map(center=tuple(roi_gdf.geometry[0].centroid.coords)[0][::-1], zoom=9)
    f = BytesIO()
    im.save(f, "png")
    data = b64encode(f.getvalue()).decode("ascii")
    imgurl = "data:image/png;base64," + data
    a,b,c,d = da.rio.bounds()
    bounds = ((d, c), (b, a))

    map.add(ImageOverlay(url = imgurl, bounds = bounds))
    map.add(
        GeoJSON(data=roi_gdf.geometry[0].__geo_interface__, style={"color": "red", "fillOpacity": 0})
    )
    display(map)

preview_miniature_on_map(nocloud_items[12])

In [ ]:
da = stackstac.stack(items)

In [ ]:
nocloud_da = da[da["eo:cloud_cover"] < 3]
nocloud_da

In [ ]:
#view = nocloud_da.sel(band="nir").isel(time=0).loc[::20, ::20]
#view.plot.imshow()

In [ ]:
view = (nocloud_da.sel(band="scl").sel(time=nocloud_da.time.values[-1]).loc[::, ::] == 5) & (nocloud_da.sel(band="scl").isel(time=0).loc[::, ::] == 4)
y,x = np.ogrid[-3:7-3, -3:7-3]
mask = x*x + y*y <= 3*3
view.values = binary_dilation(view.values, footprint=mask)
for i in range(2):
    view.values = binary_erosion(view.values, footprint=mask)
for i in range(3):
    view.values = binary_dilation(view.values, footprint=mask)
view.plot.imshow()

view.rio.write_crs(view["proj:epsg"].values).astype(np.uint8).rio.to_raster("../res/test.tif")

In [ ]:
rasterio

In [ ]:
(nocloud_da.sel(band="scl").sel(time=nocloud_da.time.values[-1]).loc[::, ::] == 6).rio.write_crs(view["proj:epsg"].values).astype(np.uint8).rio.to_raster("../res/water.tif")

In [ ]:
nocloud_da.time.values[[0, -1]]